In [1]:
import pandas as pd
import requests
import datetime
import os
import time
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

In [2]:
cities_path = r'C:\Users\jpvba_000\Documents\Eclipse_Cities.xlsx'
cities = pd.read_excel(cities_path)
cities.head()

,location,code,duration
0,Abbeville,SC,02:12:00
1,Anderson,SC,02:33:00
2,Andrews,SC,02:01:00
3,Antreville,SC,02:19:00
4,Awendaw,SC,02:33:00


In [3]:
# USER INPUT

# Provide the desired bottom limit for eclipse duration in minutes and seconds.
minutes = 2
seconds = 20

# Provide the city and corresponding state from which you will be departing.
initial_city = 'Charlottesville'
initial_state = 'VA'

In [4]:
best_cities = cities[cities['duration'] >= datetime.time(minutes, seconds)].reset_index(drop=True)
best_cities.columns = ['City', 'State', 'Eclipse Duration (m:ss)']
best_cities.head()

,City,State,Eclipse Duration (m:ss)
0,Anderson,SC,02:33:00
1,Awendaw,SC,02:33:00
2,Batesburg-Leesville,SC,02:30:00
3,Belton,SC,02:37:00
4,Bonneau,SC,02:35:00


In [5]:
# Create a function that takes in the starting city and state and returns the time and distance to drive
# to each city.

def trip_info(initial_city, initial_state_abbreviation, final_city,
              final_state_abbreviation):
    api_key = os.environ['Google_Distance_Matrix_API_key']
    
    # URL parameters
    url_friendly_initial_city = '+'.join(initial_city.split())
    url_friendly_final_city = '+'.join(final_city.split())
    origin = '{},{}'.format(url_friendly_initial_city, initial_state_abbreviation)
    destination = '{},{}'.format(url_friendly_final_city, final_state_abbreviation)
    request_URL = ('https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
                   + 'origins={}&destinations={}&key={}'
                   .format(origin, destination, api_key))
    
    # Obtain and parse response
    response = requests.get(request_URL)
    if response.status_code == 200:
        response_json = response.json()
        distance = response_json['rows'][0]['elements'][0]['distance']['text']
        duration = response_json['rows'][0]['elements'][0]['duration']['text']
        return distance,duration
    else:
        print('Could not find a viable route between these cities.')
        return None, None

In [6]:
trip_distance_and_time = best_cities.apply(lambda x: trip_info(initial_city, initial_state, 
                                                               x['City'], x['State']),
                                           axis='columns')
best_cities[['1st Leg Distance','1st Leg Duration']] = trip_distance_and_time.apply(pd.Series)
best_cities.head()

,City,State,Eclipse Duration (m:ss),1st Leg Distance,1st Leg Duration
0,Anderson,SC,02:33:00,397 mi,6 hours 19 mins
1,Awendaw,SC,02:33:00,461 mi,7 hours 5 mins
2,Batesburg-Leesville,SC,02:30:00,394 mi,6 hours 13 mins
3,Belton,SC,02:37:00,392 mi,6 hours 14 mins
4,Bonneau,SC,02:35:00,442 mi,6 hours 31 mins


In [7]:
# Obtain sample response from the Weather Underground API.
weather_api_key = os.environ['Weather_Underground_API_key']
URL_request = 'http://api.wunderground.com/api/{}/hourly10day/q/SC/Lexington.json'.format(weather_api_key)
response = requests.get(URL_request).json()

In [8]:
# The hourly_forecast key returns a list. The elements in this list change every hour.
# Obtain the index of the weather values for 10AM through 5PM for August 21.
for num, weather_info in enumerate(response['hourly_forecast']):
    if weather_info['FCTTIME']['civil'] == '10:00 AM' and weather_info['FCTTIME']['mday']=='21':
        start_num = num
        end_num = num + 8
        break
hourly_info = response['hourly_forecast'][start_num:end_num]

In [9]:
# Create columns for 10AM through 5PM
for weather_dicts in hourly_info:
    time_of_day = weather_dicts['FCTTIME']['civil']
    best_cities[time_of_day]=""

# Sort dataframe by ascending driving duration and descending eclipse duration.
best_cities.sort_values(by=['1st Leg Duration', 'Eclipse Duration (m:ss)'],
                        axis='rows',
                        ascending=[True, False],
                        inplace=True)
best_cities.reset_index(drop=True, inplace=True)
best_cities.head()

,City,State,Eclipse Duration (m:ss),1st Leg Distance,1st Leg Duration,10:00 AM,11:00 AM,12:00 PM,1:00 PM,2:00 PM,3:00 PM,4:00 PM,5:00 PM
0,St. Andrews,SC,02:29:00,364 mi,5 hours 42 mins,,,,,,,,
1,Columbia,SC,02:30:00,363 mi,5 hours 44 mins,,,,,,,,
2,Irmo,SC,02:29:00,371 mi,5 hours 48 mins,,,,,,,,
3,Peak,SC,02:21:00,358 mi,5 hours 48 mins,,,,,,,,
4,Lexington,SC,02:36:00,373 mi,5 hours 51 mins,,,,,,,,


In [10]:
# Create a function that looks up the icon and cloud cover for each location between 10AM and 5PM.
def weather_info(city, state_abbreviation, *args):
    # Args
    api_key, start, stop, *rest = args
    # URL parameters
    url_friendly_city = '_'.join(city.split())
    request_URL = (r'http://api.wunderground.com/api/{}/hourly10day/q/{}/{}.json'
                   .format(api_key, state_abbreviation, url_friendly_city))
    # Parse response
    response = requests.get(request_URL)
    if response.status_code == 200 and 'hourly_forecast' in response.json().keys():
        hourly_info = response.json()['hourly_forecast'][start:stop]
        
        # The sky key returns the cloud cover.
        return tuple(['<img src="{}" /> {}%'.format(weather['icon_url'], weather['sky']) for weather in hourly_info])
    else:
        return tuple([None for _ in range(8)])

In [11]:
# Limited to 10 calls per minute on the Weatherunderground API. Be safe.
def avoid_API_limit(dataframe, *args, call_limit=9, limit_time=65):
    time.sleep(limit_time)
    start_index = 0
    length_data = dataframe.shape[0]
    while length_data > start_index + call_limit:
        if start_index == 0:
            image_series = (dataframe.iloc[:start_index+call_limit]
                            .apply(lambda x: weather_info(x['City'], x['State']), axis='columns'))
        else:
            image_series = image_series.append((dataframe.iloc[start_index:start_index+call_limit]
                                                .apply(lambda x: weather_info(x['City'], x['State'], *args), axis='columns')),
                                               ignore_index=True)
        start_index += call_limit
        time.sleep(limit_time)
    image_series = image_series.append((dataframe.iloc[start_index:].apply(lambda x: weather_info(x['City'], x['State'], *args),axis='columns')),
                                       ignore_index=True)
    return image_series

In [12]:
city_weather = avoid_API_limit(best_cities, weather_api_key, start_num, end_num)

In [13]:
best_cities[best_cities.columns[5:]] = city_weather.apply(pd.Series)

In [14]:
HTML(best_cities.to_html(escape=False))

,City,State,Eclipse Duration (m:ss),1st Leg Distance,1st Leg Duration,10:00 AM,11:00 AM,12:00 PM,1:00 PM,2:00 PM,3:00 PM,4:00 PM,5:00 PM
0,St. Andrews,SC,02:29:00,364 mi,5 hours 42 mins,69%,56%,69%,68%,64%,68%,68%,66%
1,Columbia,SC,02:30:00,363 mi,5 hours 44 mins,69%,60%,69%,66%,63%,66%,64%,64%
2,Irmo,SC,02:29:00,371 mi,5 hours 48 mins,66%,54%,62%,56%,37%,45%,50%,52%
3,Peak,SC,02:21:00,358 mi,5 hours 48 mins,64%,57%,66%,62%,49%,44%,49%,48%
4,Lexington,SC,02:36:00,373 mi,5 hours 51 mins,66%,60%,69%,72%,71%,71%,76%,63%
5,Cayce,SC,02:33:00,372 mi,5 hours 51 mins,68%,59%,69%,69%,62%,69%,65%,65%
6,Gantt,SC,02:23:00,370 mi,5 hours 52 mins,1%,11%,21%,36%,42%,56%,48%,36%
7,Pomaria,SC,02:24:00,362 mi,5 hours 53 mins,64%,56%,64%,57%,44%,46%,48%,47%
8,Powderville,SC,02:26:00,375 mi,5 hours 54 mins,1%,11%,14%,32%,37%,46%,40%,27%
9,Little Mountain,SC,02:30:00,363 mi,5 hours 55 mins,59%,55%,63%,55%,41%,43%,47%,47%
